In [34]:
# all imports
import pandas as pd
import csv

In [35]:
def read_and_clean_df(file_name):
    
    # Read the csv file with session data, should be called results.csv
    # Example file path:
    # '/Users/oishanibandopadhyay/Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/results.csv'
    df = pd.read_csv(file_name)

    # Dropping irrelevant columns for analysis
    df.drop(['expt_id', 'group_id', 'network_error_repeat', 'participation_duration', 'response_correct', 'response_mode', 'response_rt', 'response_target', 'session_start_time', 'trial_duration', 'trial_num', 'trial_template'], axis='columns')

    # Re-order columns for better readability
    clean_df = df[['session_id', 'participant_id', 'stimuli_presented', 'response_type', 'response_name', 'response_value']]

    # Return cleaned dataframe
    return clean_df


# Test run
df = read_and_clean_df('/Users/oishanibandopadhyay/Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/results.csv')
df

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)']
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish"
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)']
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English']
...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4


In [36]:
def add_lang_bg_3(df_2):

    # first, we make a filter with the stimuli being LangBgQ3
    lang_bg_q3_filtered = df_2[df_2.loc[:, 'stimuli_presented'] == 'LangBgQ3']
    lang_bg_q3_filtered
    
    # next, we set the index to be participant_id for the mapping
    lang_bg_q3_indexed = lang_bg_q3_filtered.set_index('participant_id')
    lang_bg_q3_indexed
    
    
    # then, we get out the 'response_value' column for the mapping
    lang_bg_q3_map = lang_bg_q3_indexed['response_value']
    lang_bg_q3_map
    
    # then, we use the map function 
    df_2.loc[:, 'lang_bg_q3_resp'] = df_2.loc[:, 'participant_id'].map(lang_bg_q3_map)
    df_2
    
    # next, we're going to add a column for lang_bg_q3_english_only as a binary encoder
    df_2.loc[:, 'lang_bg_q3_english_only'] = df_2.loc[:, 'lang_bg_q3_resp'].apply(lambda x: isinstance(x, str) and 'None besides English' in x)
    df_2

    return df_2

# Test run
df = add_lang_bg_3(df)
df

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True
...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False


In [37]:
def add_prompts_from_stimulus_definitions(df):

    # read in stimulus definitions file
    stimulus_definitions = pd.read_csv('/Users/oishanibandopadhyay/Documents/Revising Honors Project/stimulus-definitions.csv')

    # drop irrelevant type column
    stimulus_definitions = stimulus_definitions.drop(['type'], axis='columns')

    # reorder and rename columns and set name as the index to later merge
    stimulus_definitions = stimulus_definitions[['name', 'content']]
    stimulus_definitions = stimulus_definitions.set_index('name')
    stimulus_definitions = stimulus_definitions.rename(columns = {'content': 'prompt_text'})

    # merge prompt_name column to the df
    df = df.merge(stimulus_definitions, left_on = 'stimuli_presented', right_index = True, how = 'left')

    return df

# Test run
df = add_prompts_from_stimulus_definitions(df)
df

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only,prompt_text
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,"Which languages, if any, do you speak besides ..."
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True,"If you selected Other, please name the other l..."
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Which languages were spoken around you where y...
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True,"If you selected Other, please name the other l..."
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True,Which languages do you think have an influence...
...,...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False,She is leaving for Savannah tomorrow.
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False,They knew where Anirudh would be on a Friday e...
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False,She is leaving for Calcutta tomorrow.
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False,"Macha what are you doing, come off to the movi..."


In [38]:
stimulus_definitions = pd.read_csv('/Users/oishanibandopadhyay/Documents/Revising Honors Project/stimulus-definitions.csv')
stimulus_definitions

,type,content,name
0,text,Hello! The following questions will ask you to...,InstructionSlide1
1,text,Please say these prompts as if you were speaki...,InstructionSlide3
2,text,Try to pronounce everything you see to the bes...,InstructionSlide2
3,text,"A set of prompts will now appear, you may clic...",BlockStartInstruction
4,text,Finding the derivative of x-dash y cos z shoul...,DialPrompt1
...,...,...,...
83,text,He is leaving for Newton tomorrow.,LocPrompt40
84,text,She is leaving for Brentwood tomorrow.,LocPrompt41
85,text,He is leaving through Golden Gate bridge tomor...,LocPrompt42
86,text,"If you have any suggestions, questions or comm...",FinalPrompt


In [39]:
stimulus_definitions = stimulus_definitions.drop(['type'], axis='columns')
stimulus_definitions = stimulus_definitions[['name', 'content']]
stimulus_definitions = stimulus_definitions.set_index('name')
stimulus_definitions = stimulus_definitions.rename(columns = {'content': 'prompt_text'})
stimulus_definitions

,prompt_text
name,
InstructionSlide1,Hello! The following questions will ask you to...
InstructionSlide3,Please say these prompts as if you were speaki...
InstructionSlide2,Try to pronounce everything you see to the bes...
BlockStartInstruction,"A set of prompts will now appear, you may clic..."
DialPrompt1,Finding the derivative of x-dash y cos z shoul...
...,...
LocPrompt40,He is leaving for Newton tomorrow.
LocPrompt41,She is leaving for Brentwood tomorrow.
LocPrompt42,He is leaving through Golden Gate bridge tomor...


In [40]:
# next, we want to map name, which is the index for stimuli
# to stimuli_presented from df

df = df.merge(stimulus_definitions, left_on='stimuli_presented', right_index=True, how='left')
df

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only,prompt_text_x,prompt_text_y
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,"Which languages, if any, do you speak besides ...","Which languages, if any, do you speak besides ..."
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True,"If you selected Other, please name the other l...","If you selected Other, please name the other l..."
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Which languages were spoken around you where y...,Which languages were spoken around you where y...
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True,"If you selected Other, please name the other l...","If you selected Other, please name the other l..."
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True,Which languages do you think have an influence...,Which languages do you think have an influence...
...,...,...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False,She is leaving for Savannah tomorrow.,She is leaving for Savannah tomorrow.
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False,They knew where Anirudh would be on a Friday e...,They knew where Anirudh would be on a Friday e...
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False,She is leaving for Calcutta tomorrow.,She is leaving for Calcutta tomorrow.
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False,"Macha what are you doing, come off to the movi...","Macha what are you doing, come off to the movi..."


In [41]:
'''Old testing building code





'''

df_1 = pd.read_csv('/Users/oishanibandopadhyay/Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/results.csv')
df_1

,expt_id,group_id,network_error_repeat,participant_id,participation_duration,response_correct,response_mode,response_name,response_rt,response_target,response_type,response_value,session_id,session_start_time,stimuli_presented,trial_duration,trial_num,trial_template
0,645895940a130c4b8be8e6c6,default,0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,1129.322649,False,click,LangBgOptionsResp,11352.0,[],choice-response,['Other (you may specify in the next slide)'],67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LangBgQ1,11569,1,LangBgSlide1
1,645895940a130c4b8be8e6c6,default,0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,1129.322649,NaN,keypress,LangBgOtherResp,8043.0,NaN,text-response,"Korean, Spanish",67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LangBgQOther,8244,2,LangBgSlide1
2,645895940a130c4b8be8e6c6,default,0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,1129.322649,False,click,LangBgOptionsResp,11273.0,[],choice-response,['Other (you may specify in the next slide)'],67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LangBgQ2,11277,3,LangBgSlide2
3,645895940a130c4b8be8e6c6,default,0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,1129.322649,NaN,keypress,LangBgOtherResp,2954.0,NaN,text-response,Korean,67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LangBgQOther,2957,4,LangBgSlide2
4,645895940a130c4b8be8e6c6,default,0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,1129.322649,False,click,LangBgOptionsResp,10716.0,[],choice-response,['None besides English'],67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LangBgQ3,10719,5,LangBgSlide3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,645895940a130c4b8be8e6c6,default,0,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,928.020371,NaN,voice,LocPromptResp39,NaN,NaN,audio-response,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LocPrompt39,4401,81,RandomBlock4
251,645895940a130c4b8be8e6c6,default,0,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,928.020371,NaN,voice,LocPromptResp22,NaN,NaN,audio-response,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LocPrompt22,8428,82,RandomBlock4
252,645895940a130c4b8be8e6c6,default,0,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,928.020371,NaN,voice,LocPromptResp29,NaN,NaN,audio-response,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LocPrompt29,4518,83,RandomBlock4
253,645895940a130c4b8be8e6c6,default,0,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,928.020371,NaN,voice,LocPromptResp27,NaN,NaN,audio-response,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,67d94e1a42f12a9364065df2,2025-03-18 06:42:34-04:00,LocPrompt27,6543,84,RandomBlock4


In [42]:
df_1.drop(['expt_id', 'group_id', 'network_error_repeat', 'participation_duration', 'response_correct', 'response_mode', 'response_rt', 'response_target', 'session_start_time', 'trial_duration', 'trial_num', 'trial_template'], axis='columns')

,participant_id,response_name,response_type,response_value,session_id,stimuli_presented
0,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgOptionsResp,choice-response,['Other (you may specify in the next slide)'],67d94e1a42f12a9364065df2,LangBgQ1
1,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgOtherResp,text-response,"Korean, Spanish",67d94e1a42f12a9364065df2,LangBgQOther
2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgOptionsResp,choice-response,['Other (you may specify in the next slide)'],67d94e1a42f12a9364065df2,LangBgQ2
3,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgOtherResp,text-response,Korean,67d94e1a42f12a9364065df2,LangBgQOther
4,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgOptionsResp,choice-response,['None besides English'],67d94e1a42f12a9364065df2,LangBgQ3
...,...,...,...,...,...,...
250,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPromptResp39,audio-response,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,67d94e1a42f12a9364065df2,LocPrompt39
251,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPromptResp22,audio-response,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,67d94e1a42f12a9364065df2,LocPrompt22
252,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPromptResp29,audio-response,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,67d94e1a42f12a9364065df2,LocPrompt29
253,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPromptResp27,audio-response,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,67d94e1a42f12a9364065df2,LocPrompt27


In [43]:
# ok so the next step needs to be something like a audio_user_id, where you take out
# just the pre-hyphen part of the audio file name
# 'ef360fc757c3a1d77fdf1b72-63-LocPromptResp4.mp4'
# 'ef360fc757c3a1d77fdf1b72' this part

#wait we just need the speech to text functions to be applied here, each response_value
# audio file w same name just needs to be transcribed and the transcription
# added as a column value, with the text also being stored as a txt file in the
# big folder overall? like a full audio pathname but txt file? 
# i feel like maybe we just make one folder called txt files and dump them
# all in there one after another

In [44]:
df_2 = df_1[['session_id', 'participant_id', 'stimuli_presented', 'response_type', 'response_name', 'response_value']]
df_2

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)']
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish"
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)']
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English']
...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4


In [45]:
'''print(df_2['stimuli_presented'])
print(type(df_2['stimuli_presented']))
for i in range(len(df_2['stimuli_presented'])):
    print(i, df_2['stimuli_presented'][i])'''

# ok so basically
# to access all the values in a column, you can index it as 
# for i in range(len(df['col_name'])):
#     df['col_name'][i]
# or you can directly iterate through without indexes as well
# think of this as a list of dictionaries with lists as values for 
# each column as a key

#print(df_2['stimuli_presented'] == 'LangBgQ3')
#filtered_df = df_2[df_2['stimuli_presented'] == 'LangBgQ3']
#print(filtered_df)
#filtered_df

#full_filtered_df = filtered_df[['participant_id', 'response_value']]
#full_filtered_df
#df_2[df_2[df_2['stimuli_presented'] == 'LangBgQ3'][['participant_id', 'response_value']]]


"print(df_2['stimuli_presented'])\nprint(type(df_2['stimuli_presented']))\nfor i in range(len(df_2['stimuli_presented'])):\n    print(i, df_2['stimuli_presented'][i])"

In [46]:
#lang_bg_q3_map = df_2[df_2.loc[:, 'stimuli_presented'] == 'LangBgQ3'].set_index('participant_id')['response_value']
#lang_bg_q3_map

# ok so this is made up of a few parts
# first, we make a filter with the stimuli being correct
lang_bg_q3_filtered = df_2[df_2.loc[:, 'stimuli_presented'] == 'LangBgQ3']
lang_bg_q3_filtered

# next, we set the index to be participant_id for the mapping
lang_bg_q3_indexed = lang_bg_q3_filtered.set_index('participant_id')
lang_bg_q3_indexed


# then, we get out the 'response_value' column for the mapping
lang_bg_q3_map = lang_bg_q3_indexed['response_value']
lang_bg_q3_map

# then, we use the map function 
df_2.loc[:, 'lang_bg_q3_resp'] = df_2.loc[:, 'participant_id'].map(lang_bg_q3_map)
df_2

# next, we're going to add a column for lang_bg_q3_english_only as a binary encoder
df_2.loc[:, 'lang_bg_q3_english_only'] = df_2.loc[:, 'lang_bg_q3_resp'].apply(lambda x: isinstance(x, str) and 'None besides English' in x)
df_2

/var/folders/0_/jxh577k50pn0gjfk0m3vz0ch0000gn/T/ipykernel_60064/3548627651.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'lang_bg_q3_resp'] = df_2.loc[:, 'participant_id'].map(lang_bg_q3_map)
/var/folders/0_/jxh577k50pn0gjfk0m3vz0ch0000gn/T/ipykernel_60064/3548627651.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'lang_bg_q3_english_only'] = df_2.loc[:, 'lang_bg_q3_resp'].apply(lambda x: isinstance(x, str) and 'None besides English' in x)


,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True
...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False


In [47]:
lang_bg_q3_map = df_2[df_2.loc[:, 'stimuli_presented'] == 'LangBgQ3'].set_index('participant_id')['response_value']
df_2.loc[:,'lang_bg_q3_resp'] = df_2.loc[:, 'participant_id'].map(lang_bg_q3_map)
df_2

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True
...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False


In [48]:
df_2.loc[:, 'lang_bg_q3_english_only'] = df_2.loc[:, 'lang_bg_q3_resp'].apply(lambda x: isinstance(x, str) and 'None besides English' in x)
df_2

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True
...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False


In [49]:
df_3 = df_2

In [50]:
df_3.head()

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True


In [51]:
df_3.loc[:, 'trial_format'] = df_3.apply(lambda row: 'Documents/Revising Honors Project/session-{session_id}-data/audio/{response_value}'.format(session_id = row['session_id'], response_value = row['response_value']), axis=1)
df_3
df_3['trial_format'][0:5] 

/var/folders/0_/jxh577k50pn0gjfk0m3vz0ch0000gn/T/ipykernel_60064/2835670782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3.loc[:, 'trial_format'] = df_3.apply(lambda row: 'Documents/Revising Honors Project/session-{session_id}-data/audio/{response_value}'.format(session_id = row['session_id'], response_value = row['response_value']), axis=1)


0    Documents/Revising Honors Project/session-67d9...
1    Documents/Revising Honors Project/session-67d9...
2    Documents/Revising Honors Project/session-67d9...
3    Documents/Revising Honors Project/session-67d9...
4    Documents/Revising Honors Project/session-67d9...
Name: trial_format, dtype: object

In [52]:
if 'mp4' in df_3.loc[:, 'response_value']:
    print('hi')

print(df_3.loc[:, 'response_value'])

0        ['Other (you may specify in the next slide)']
1                                      Korean, Spanish
2        ['Other (you may specify in the next slide)']
3                                               Korean
4                             ['None besides English']
                            ...                       
250    53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4
251    53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4
252    53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4
253    53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4
254                                                NaN
Name: response_value, Length: 255, dtype: object


In [53]:
#df_3.loc[:, 'trial_format'] = df_3.apply(lambda row: 'Documents/Revising Honors Project/session-{session_id}-data/audio/{response_value}'.format(session_id = row['session_id'], response_value = row['response_value']), axis=1)

df_3.loc[:, 'trial_function'] = df_3.apply(lambda x: f'{x['trial_format']}' if isinstance(x['trial_format'], str) and 'mp4' in x['trial_format'] else None, axis=1)
df_3                                                                                      

/var/folders/0_/jxh577k50pn0gjfk0m3vz0ch0000gn/T/ipykernel_60064/2214632609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3.loc[:, 'trial_function'] = df_3.apply(lambda x: f'{x['trial_format']}' if isinstance(x['trial_format'], str) and 'mp4' in x['trial_format'] else None, axis=1)


,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only,trial_format,trial_function
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
...,...,...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...


In [54]:
df_2

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only,trial_format,trial_function
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
...,...,...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...


In [55]:
df_3.loc[:, 'trial_format'] = df_3.apply(lambda row: 'Documents/Revising Honors Project/session-{session_id}-data/audio/{response_value}'.format(session_id = row['session_id'], response_value = row['response_value']), axis=1)
df_3.loc[:, 'trial_function'] = df_3.apply(lambda x: f'{x['trial_format']}' if isinstance(x['trial_format'], str) and 'mp4' in x['trial_format'] else None, axis=1)

In [56]:
df_3

,session_id,participant_id,stimuli_presented,response_type,response_name,response_value,lang_bg_q3_resp,lang_bg_q3_english_only,trial_format,trial_function
0,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ1,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
1,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,"Korean, Spanish",['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
2,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ2,choice-response,LangBgOptionsResp,['Other (you may specify in the next slide)'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
3,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQOther,text-response,LangBgOtherResp,Korean,['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
4,67d94e1a42f12a9364065df2,b5bdd1b55fdf1a07a1cc3907eb6ade997ee127e8708b58...,LangBgQ3,choice-response,LangBgOptionsResp,['None besides English'],['None besides English'],True,Documents/Revising Honors Project/session-67d9...,None
...,...,...,...,...,...,...,...,...,...,...
250,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt39,audio-response,LocPromptResp39,53c7dbc41f259d6f7157d364-81-LocPromptResp39.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
251,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt22,audio-response,LocPromptResp22,53c7dbc41f259d6f7157d364-82-LocPromptResp22.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
252,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt29,audio-response,LocPromptResp29,53c7dbc41f259d6f7157d364-83-LocPromptResp29.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...
253,67d94e1a42f12a9364065df2,9e4c7d9ba7377ea14d0e228c38df883deaf62f3f4d1837...,LocPrompt27,audio-response,LocPromptResp27,53c7dbc41f259d6f7157d364-84-LocPromptResp27.mp4,['Other (you may specify in the next slide)'],False,Documents/Revising Honors Project/session-67d9...,Documents/Revising Honors Project/session-67d9...


In [57]:
list_trial_format = df_3['trial_format']

In [58]:
list_trial_format

0      Documents/Revising Honors Project/session-67d9...
1      Documents/Revising Honors Project/session-67d9...
2      Documents/Revising Honors Project/session-67d9...
3      Documents/Revising Honors Project/session-67d9...
4      Documents/Revising Honors Project/session-67d9...
                             ...                        
250    Documents/Revising Honors Project/session-67d9...
251    Documents/Revising Honors Project/session-67d9...
252    Documents/Revising Honors Project/session-67d9...
253    Documents/Revising Honors Project/session-67d9...
254    Documents/Revising Honors Project/session-67d9...
Name: trial_format, Length: 255, dtype: object

In [59]:
for i in range(0,10):
    print(list_trial_format[i])

Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/['Other (you may specify in the next slide)']
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/Korean, Spanish
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/['Other (you may specify in the next slide)']
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/Korean
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/['None besides English']
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/nan
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/['None of these']
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/nan
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/['Other (you may specify in the next slide)']
Documents/Revising Honors Project/session-67d94e1a42f12a9364065df2-data/audio/Korean
